# Codex & MCP: Sharing for Scalability
Santa's Workshop Inventory demo deck


## Goal for this deck
- Show how MCP server sharing improves consistency and resource use
- Provide runnable cues for the Santa's Workshop Inventory server
- Walk through separate vs shared modes you can demo live


## Case 1: Separate mode (two servers)
- Two HTTP MCP servers: 127.0.0.1:6274 and 127.0.0.1:6275
- Elf A client binds to Server A; Elf B client binds to Server B
- Duplicate processes, duplicate in-memory inventory state


## Separate mode: runbook
```bash
source .venv/bin/activate
python run_http_server.py --port 6274 --path /mcp  # Server A
python run_http_server.py --port 6275 --path /mcp  # Server B
```
Client config examples:
```toml
[mcp_servers."workshop-elf-a"]
transport = "http"
url = "http://127.0.0.1:6274/mcp"

[mcp_servers."workshop-elf-b"]
transport = "http"
url = "http://127.0.0.1:6275/mcp"
```
Prompt cues:
- `Hello, I am Elf A. Use the workshop tools to check the current stock of "Toy Train".`
- `Hello, I am Elf B. Use the workshop tools to check the current stock of "Toy Train".`


## Separate mode: what you see
- Both elves read Toy Train stock: 50
- Elf A produces 10 Toy Trains; Server A now reports 60
- Elf B re-checks: still 50 (its server never saw the update)
- Outcomes: inconsistent data + wasted CPU/RAM from duplicate servers


## Case 2: Shared mode (one server)
- One HTTP MCP server at 127.0.0.1:6274 serving all clients
- Both Elves point to the same URL; inventory state is shared
- Less resource use and consistent results for every client


## Shared mode: runbook
```bash
source .venv/bin/activate
python run_http_server.py --port 6274 --path /mcp  # Single shared server
```
Client config (both Elves share):
```toml
[mcp_servers."santas-workshop-inventory"]
transport = "http"
url = "http://127.0.0.1:6274/mcp"
```
Prompt cues:
- `Elf A reporting in. Use the workshop tools to check the current stock of "Toy Train".`
- `Elf B reporting in. Use the workshop tools to check the current stock of "Toy Train".`
- `Produce 5 "Doll"s using the produce_toys tool.`
- `Produce 10 "Teddy Bear"s using the produce_toys tool.`


## Shared mode: results
- Both elves see Toy Train stock: 60 (single source of truth)
- Concurrent production hits the same server:
  - Elf A makes 5 Dolls -> stock 205
  - Elf B makes 10 Teddy Bears -> stock 110
- Outcomes: consistent data, less operational overhead, scalable pattern


## Takeaways and talking points
- Separate servers isolate context but fragment state and waste resources
- A shared MCP server keeps AI clients synchronized and cheaper to run
- Reuse this pattern for any tool-heavy workflow that multiple agents need
- Demo flow: start servers, run prompts above, call out the contrasting results
